<a href="https://colab.research.google.com/github/dallen29/Basic-Portfolio/blob/master/Copy_of_SolarInspectionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import numpy as np
import os

# Base path where the folders are located
base_dir = '/content/drive/MyDrive/InfraredSolarModules/images'

# List of folder names, corresponding to each class
class_folders = [
    "Cell", "Cell-Multi", "Cracking", "Diode", "Diode-Multi",
    "Hot-Spot", "Hot-Spot-Multi", "No-Anomaly", "Offline-Module",
    "Shadowing", "Soiling", "Vegetation"
]

images = []
labels = []
label_to_index = {class_name: index for index, class_name in enumerate(class_folders)}
index_to_label = {index: class_name for class_name, index in label_to_index.items()}

# Process each folder and load the images
for class_name in class_folders:
    class_dir = os.path.join(base_dir, class_name)
    for img_name in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_name)
        try:
            with Image.open(img_path) as img:
                img = img.resize((224, 224))  # Adjust based on your model input size
                img_array = np.asarray(img)
                images.append(img_array)
                labels.append(label_to_index[class_name])
        except IOError:
            # This handles cases where the file cannot be opened as an image
            print(f"Error opening {img_path}")

images = np.array(images, dtype='float32') / 255.0  # Normalize pixel values
labels = np.array(labels, dtype='int')

# Convert labels to one-hot encoding
num_classes = len(class_folders)
labels = np.eye(num_classes)[labels]

print(f"Total images loaded: {len(images)}")
print(f"Total labels processed: {len(labels)}")

Total images loaded: 1003
Total labels processed: 1003


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers import Adam

# Load the pre-trained base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers to prevent them from being updated during training
base_model.trainable = False

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dense(len(class_folders), activation='softmax')  # Output layer with one neuron per class
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

94765736/94765736 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 dense_1 (Dense)             (None, 12)                12300     
                                                                 
Total params: 25698188 (98.03 MB)
Trainable params: 2110476 (8.05 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_dir = '/content/drive/MyDrive/InfraredSolarModules/images'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2)  # using 20% of the data for validation

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),  # Resize images to this target size
    batch_size=32,
    class_mode='categorical',  # for multi-class classification
    subset='training')  # specify this is training data

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')  # specify this is validation data

Found 1264 images belonging to 12 classes.
Found 315 images belonging to 12 classes.


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')  # output layer with a node for each class
])


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')  # output layer with a node for each class
])


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')  # output layer with a node for each class
])


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,  # Number of batches to process per epoch
    epochs=10,  # Start with a small number of epochs; adjust based on your dataset size and complexity
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)


Epoch 1/10
39/39 [==============================] - 229s 6s/step - loss: 0.9837 - accuracy: 0.5073 - val_loss: 0.7375 - val_accuracy: 0.4722
Epoch 2/10
39/39 [==============================] - 201s 5s/step - loss: 0.7119 - accuracy: 0.5308 - val_loss: 0.6792 - val_accuracy: 0.5903
Epoch 3/10
39/39 [==============================] - 197s 5s/step - loss: 0.6745 - accuracy: 0.5812 - val_loss: 0.6636 - val_accuracy: 0.5868
Epoch 4/10
39/39 [==============================] - 198s 5s/step - loss: 0.6679 - accuracy: 0.5990 - val_loss: 0.6476 - val_accuracy: 0.6111
Epoch 5/10
39/39 [==============================] - 189s 5s/step - loss: 0.6447 - accuracy: 0.6177 - val_loss: 0.6643 - val_accuracy: 0.5938
Epoch 6/10
39/39 [==============================] - 197s 5s/step - loss: 0.6592 - accuracy: 0.6201 - val_loss: 0.6407 - val_accuracy: 0.6215
Epoch 7/10
39/39 [==============================] - 198s 5s/step - loss: 0.6504 - accuracy: 0.6461 - val_loss: 0.6330 - val_accuracy: 0.6042
Epoch 8/10
39

In [ ]:
!pip install numpy scikit-learn matplotlib


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np
import matplotlib.pyplot as plt

# Assuming `X_test` and `y_test` are your features and labels for the test set respectively
# And `model` is your trained Keras model

# Predict the labels for the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)  # Only necessary if y_test is one-hot encoded

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred_classes)

# Generate a classification report
class_report = classification_report(y_true, y_pred_classes)

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)

# Print the summary
print(f"Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:\n", class_report)

# Function to plot confusion matrix
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in np.ndenumerate(cm):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Assuming you have an array `class_names` with the names of the classes
plot_confusion_matrix(conf_matrix, classes=class_names, title='Confusion Matrix')


NameError: name 'X_test' is not defined